# Periodic Boundary Condition

An explained implementation.

Some searching reveals this [blogpost](http://blog.lostinmyterminal.com/physics/2015/04/09/periodic-boundary-subtraction.html) which has the four functions:

In [1]:
import numpy as np

In [2]:
def dists(rs, L):
    """
    Returns the difference vectors between all pairs of points in rs,
    given a periodic box of size L.

    Parameters
    ----------
    rs : (N,d) array_like
         Input vectors
    L : (d,) array_like, or scalar
    Size of the box

    Returns
    -------
    out : (N,N,d) ndarray
    Where out[i,j,:] is the difference vector between rs[i,:] and rs[j,:]
    """
    diffs = np.array([np.subtract.outer(rd, rd) for rd in rs.T]).T
    return np.remainder(diffs + L/2., L) - L/2.

def dists3(rs, L):
    diffs = rs[..., np.newaxis] - rs.T[np.newaxis, ...]
    diffs = np.rollaxis(diffs, -1, -2)
    return np.remainder(diffs + L/2., L) - L/2.

def pair_dists(rs1, rs2, L):
#     N,d = np.shape(rs) Broken line! There is no N, d....
    diffs = rs1[..., np.newaxis] - rs2.T[np.newaxis, ...]
    diffs = np.rollaxis(diffs, -1, -2)
    return np.remainder(diffs + L/2., L) - L/2.

def pair_dist(rs1, rs2, L):
    """
    Returns the total distance between pairs of points in rs1 and rs2, given a 
    periodic box of size L. Ignores overall translation.
    """
    dists1 = (rs1[..., np.newaxis] - rs1.T[np.newaxis, ...])
    dists2 = (rs2[..., np.newaxis] - rs2.T[np.newaxis, ...])
    rij_minus_sij = np.rollaxis(dists1 - dists2, -1, -2)
    rij_minus_sij = ((rij_minus_sij + L/2.) % L) - L/2.
    dist_sqs = np.triu(np.sum(rij_minus_sij**2, axis=-1))
    return np.sqrt(np.sum(dist_sqs) / len(rs1))

It is not extraordinarily clear what is going on in the above functinos. Also some of the `numpy` functions called are depreciated (rollaxis) and should be replaced...

The use of ellipsis `...` in `numpy` is defined:

> Ellipsis expand to the number of `:` objects needed to make a selection tuple of the same length as `x.ndim`. There may only be a single ellipsis present.

In [3]:
x = np.array([[[1],[2],[3]], [[4],[5],[6]]])
x

array([[[1],
        [2],
        [3]],

       [[4],
        [5],
        [6]]])

In [4]:
x[:, 0]

array([[1],
       [4]])

In [5]:
x[:,:, 0]

array([[1, 2, 3],
       [4, 5, 6]])

In [6]:
x[..., 0]

array([[1, 2, 3],
       [4, 5, 6]])

## Examining the first function

```python
def dists(rs, L):
    diffs = np.array([np.subtract.outer(rd, rd) for rd in rs.T]).T
    return np.remainder(diffs + L/2., L) - L/2.
```

In [7]:
xyz_array = np.array([
    [0, 0, 0],
    [1, 1, 1],
    [1, 2, 3],
    [1, 2, 2],
    [0, 0, 1],
    [2, 3, 4],
])
xyz_array

array([[0, 0, 0],
       [1, 1, 1],
       [1, 2, 3],
       [1, 2, 2],
       [0, 0, 1],
       [2, 3, 4]])

In [8]:
# Examine the function from inside out.
def explicit_dists(point_array, period_length=2.0):
    
    print(f'starting input array:\n{point_array}\n')
    
    # Make a list to hold difference arrays:
    diff_list = list()
    
    # Get the transpose of the point array.
    # So we will iterate through the X, Y, and Z portions of
    # the input array.
    for pa in point_array.T:
        
        print(f'An X, Y, or Z coordinate array:\n{pa}')
        
        # The .outer function applies the leading function,
        # in this case a subtraction, to all possible pairs
        # with the two arrays given.
        # https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ufunc.outer.html
        diff_array = np.subtract.outer(pa, pa)
        diff_list.append(diff_array)
        print(f'Difference array:\n{diff_array}\n')
        
    # Create an array from the distances.
    diffs = np.array(diff_list)
    print(f'Complete diff array:\n{diffs}')
    
    difft = diffs.T
    print(f'Transposed diff array:\n{difft}')
    
    remainder = np.remainder(difft + period_length / 2.0, period_length) - period_length / 2.0
    print(f'remainder array:\n{remainder}\n')
    
    return remainder

In [9]:
explicit_dists(xyz_array)

starting input array:
[[0 0 0]
 [1 1 1]
 [1 2 3]
 [1 2 2]
 [0 0 1]
 [2 3 4]]

An X, Y, or Z coordinate array:
[0 1 1 1 0 2]
Difference array:
[[ 0 -1 -1 -1  0 -2]
 [ 1  0  0  0  1 -1]
 [ 1  0  0  0  1 -1]
 [ 1  0  0  0  1 -1]
 [ 0 -1 -1 -1  0 -2]
 [ 2  1  1  1  2  0]]

An X, Y, or Z coordinate array:
[0 1 2 2 0 3]
Difference array:
[[ 0 -1 -2 -2  0 -3]
 [ 1  0 -1 -1  1 -2]
 [ 2  1  0  0  2 -1]
 [ 2  1  0  0  2 -1]
 [ 0 -1 -2 -2  0 -3]
 [ 3  2  1  1  3  0]]

An X, Y, or Z coordinate array:
[0 1 3 2 1 4]
Difference array:
[[ 0 -1 -3 -2 -1 -4]
 [ 1  0 -2 -1  0 -3]
 [ 3  2  0  1  2 -1]
 [ 2  1 -1  0  1 -2]
 [ 1  0 -2 -1  0 -3]
 [ 4  3  1  2  3  0]]

Complete diff array:
[[[ 0 -1 -1 -1  0 -2]
  [ 1  0  0  0  1 -1]
  [ 1  0  0  0  1 -1]
  [ 1  0  0  0  1 -1]
  [ 0 -1 -1 -1  0 -2]
  [ 2  1  1  1  2  0]]

 [[ 0 -1 -2 -2  0 -3]
  [ 1  0 -1 -1  1 -2]
  [ 2  1  0  0  2 -1]
  [ 2  1  0  0  2 -1]
  [ 0 -1 -2 -2  0 -3]
  [ 3  2  1  1  3  0]]

 [[ 0 -1 -3 -2 -1 -4]
  [ 1  0 -2 -1  0 -3]
  [ 3  2  0  

array([[[ 0.,  0.,  0.],
        [-1., -1., -1.],
        [-1.,  0., -1.],
        [-1.,  0.,  0.],
        [ 0.,  0., -1.],
        [ 0., -1.,  0.]],

       [[-1., -1., -1.],
        [ 0.,  0.,  0.],
        [ 0., -1.,  0.],
        [ 0., -1., -1.],
        [-1., -1.,  0.],
        [-1.,  0., -1.]],

       [[-1.,  0., -1.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0., -1.],
        [-1.,  0.,  0.],
        [-1., -1., -1.]],

       [[-1.,  0.,  0.],
        [ 0., -1., -1.],
        [ 0.,  0., -1.],
        [ 0.,  0.,  0.],
        [-1.,  0., -1.],
        [-1., -1.,  0.]],

       [[ 0.,  0., -1.],
        [-1., -1.,  0.],
        [-1.,  0.,  0.],
        [-1.,  0., -1.],
        [ 0.,  0.,  0.],
        [ 0., -1., -1.]],

       [[ 0., -1.,  0.],
        [-1.,  0., -1.],
        [-1., -1., -1.],
        [-1., -1.,  0.],
        [ 0., -1., -1.],
        [ 0.,  0.,  0.]]])

# Desire two different functions

We want one function for creating the periodic distance matrix, and one to calculate the euclidean distance. Let us examine the next function -- which is much clearer now -- it returns the distance vectors between two sets of points.

In [44]:
def periodic_dist_vector(psa, psb, pL):
    diff_arrays = psa[..., None] - psb.T[None, ...]
    diff_arrays = np.moveaxis(diff_arrays, -1, 0)
    return np.remainder(diff_arrays + pL / 2.0, pL) - pL / 2.0

In [45]:
expl_pair_dist(xyz_array, xyz_array, 2.0)

array([[[ 0.,  0.,  0.],
        [-1., -1., -1.],
        [-1.,  0., -1.],
        [-1.,  0.,  0.],
        [ 0.,  0., -1.],
        [ 0., -1.,  0.]],

       [[-1., -1., -1.],
        [ 0.,  0.,  0.],
        [ 0., -1.,  0.],
        [ 0., -1., -1.],
        [-1., -1.,  0.],
        [-1.,  0., -1.]],

       [[-1.,  0., -1.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0., -1.],
        [-1.,  0.,  0.],
        [-1., -1., -1.]],

       [[-1.,  0.,  0.],
        [ 0., -1., -1.],
        [ 0.,  0., -1.],
        [ 0.,  0.,  0.],
        [-1.,  0., -1.],
        [-1., -1.,  0.]],

       [[ 0.,  0., -1.],
        [-1., -1.,  0.],
        [-1.,  0.,  0.],
        [-1.,  0., -1.],
        [ 0.,  0.,  0.],
        [ 0., -1., -1.]],

       [[ 0., -1.,  0.],
        [-1.,  0., -1.],
        [-1., -1., -1.],
        [-1., -1.,  0.],
        [ 0., -1., -1.],
        [ 0.,  0.,  0.]]])

In [66]:
xyz_array

array([[0, 0, 0],
       [1, 1, 1],
       [1, 2, 3],
       [1, 2, 2],
       [0, 0, 1],
       [2, 3, 4]])

In [63]:
def euclidean_dist(dist_vector_array):
    """Calculate the euclidean distance of the value.
    """
    return np.linalg.norm(dist_vector_array, axis=1)

In [64]:
print(euclidean_dist(xyz_array))

[ 0.          1.73205081  3.74165739  3.          1.          5.38516481]


In [13]:
def distance(pa, pb):
    """Non-periodic distance function for comparission."""
    return np.linalg.norm(pa - pb)

In [14]:
distance(xyz_array[0], xyz_array[1])

1.7320508075688772

In [15]:
for p in xyz_array.T:
    print(p)

[0 1 1 1 0 2]
[0 1 2 2 0 3]
[0 1 3 2 1 4]
